In [ ]:
# AI Readiness Analysis: South Asia Regional Model
# Train on: Afghanistan, Bangladesh, Bhutan, India, Maldives, Pakistan, Sri Lanka
# Test on: Nepal (2023-2024)
# Predict Nepal 2025 (with 10% improvements)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error


# Load the datasets
df_2021 = pd.read_excel("/kaggle/input/government-ai-readiness-index-20212024/2021-Government-AI-Readiness-Index-public-dataset.xlsx")
df_2022 = pd.read_excel("/kaggle/input/government-ai-readiness-index-20212024/2022-Government-AI-Readiness-Index-public-data.xlsx")
df_2023 = pd.read_excel("/kaggle/input/government-ai-readiness-index-20212024/2023-Government-AI-Readiness-Index-Public-Indicator-Data.xlsx")
df_2024 = pd.read_excel("/kaggle/input/government-ai-readiness-index-20212024/2024-GAIRI-data.xlsx")

print("Data loaded successfully!")
print(f"2021 columns: {df_2021.columns.tolist()}")
print(f"2022 columns: {df_2022.columns.tolist()}")
print(f"2023 columns: {df_2023.columns.tolist()}")
print(f"2024 columns: {df_2024.columns.tolist()}")

# Based on your table, the column names are:
# Country, Overall score, Government Score, Data and Infrastructure, Vision, 
# Governance and Ethics, Digital Capacity, Adaptability, Size, Innovation Capacity,
# Human Capital, Infrastructure, Data Availability, Data Accessibility

# Let's check if "Overall score" exists, otherwise try "Overall"
overall_col_2021 = "Overall score" if "Overall score" in df_2021.columns else "Overall"
overall_col_2022 = "Overall score" if "Overall score" in df_2022.columns else "Overall"
overall_col_2023 = "Overall score" if "Overall score" in df_2023.columns else "Overall"
overall_col_2024 = "Overall score" if "Overall score" in df_2024.columns else "Overall"

country_col = "Country"  # This should be consistent across all datasets

print(f"\nUsing columns: Country='{country_col}', Overall='{overall_col_2021}' (2021)")

# Find common countries across all datasets
common_countries = set(df_2021[country_col].dropna().astype(str).str.strip())
for df in [df_2022, df_2023, df_2024]:
    common_countries = common_countries.intersection(set(df[country_col].dropna().astype(str).str.strip()))

print(f"\nNumber of common countries: {len(common_countries)}")

# Filter and rename columns for each year
df_2021_filtered = df_2021[df_2021[country_col].astype(str).str.strip().isin(common_countries)][[country_col, overall_col_2021]].copy()
df_2022_filtered = df_2022[df_2022[country_col].astype(str).str.strip().isin(common_countries)][[country_col, overall_col_2022]].copy()
df_2023_filtered = df_2023[df_2023[country_col].astype(str).str.strip().isin(common_countries)][[country_col, overall_col_2023]].copy()
df_2024_filtered = df_2024[df_2024[country_col].astype(str).str.strip().isin(common_countries)][[country_col, overall_col_2024]].copy()

# Rename columns
df_2021_filtered = df_2021_filtered.rename(columns={overall_col_2021: "Overall_2021"})
df_2022_filtered = df_2022_filtered.rename(columns={overall_col_2022: "Overall_2022"})
df_2023_filtered = df_2023_filtered.rename(columns={overall_col_2023: "Overall_2023"})
df_2024_filtered = df_2024_filtered.rename(columns={overall_col_2024: "Overall_2024"})

# Standardize country column name
df_2021_filtered = df_2021_filtered.rename(columns={country_col: "Country"})
df_2022_filtered = df_2022_filtered.rename(columns={country_col: "Country"})
df_2023_filtered = df_2023_filtered.rename(columns={country_col: "Country"})
df_2024_filtered = df_2024_filtered.rename(columns={country_col: "Country"})

# Clean country names (strip whitespace)
for df in [df_2021_filtered, df_2022_filtered, df_2023_filtered, df_2024_filtered]:
    df["Country"] = df["Country"].astype(str).str.strip()

# Merge all dataframes
merged_df = df_2021_filtered.merge(df_2022_filtered, on="Country", how="inner")
merged_df = merged_df.merge(df_2023_filtered, on="Country", how="inner")
merged_df = merged_df.merge(df_2024_filtered, on="Country", how="inner")

print(f"\nMerged dataframe shape: {merged_df.shape}")

# Create the consolidated Excel file
with pd.ExcelWriter("AI_Readiness_Index_2021_2024_Consolidated.xlsx", engine='openpyxl') as writer:
    # 1. Consolidated sheet with overall scores
    merged_df.to_excel(writer, sheet_name='Consolidated_Overall', index=False)
    
    # 2. Detailed 2024 data for common countries
    df_2024_common = df_2024[df_2024[country_col].astype(str).str.strip().isin(common_countries)]
    df_2024_common.to_excel(writer, sheet_name='Detailed_2024', index=False)
    
    # 3. Detailed 2023 data for common countries
    df_2023_common = df_2023[df_2023[country_col].astype(str).str.strip().isin(common_countries)]
    df_2023_common.to_excel(writer, sheet_name='Detailed_2023', index=False)
    
    # 4. Detailed 2022 data for common countries
    df_2022_common = df_2022[df_2022[country_col].astype(str).str.strip().isin(common_countries)]
    df_2022_common.to_excel(writer, sheet_name='Detailed_2022', index=False)
    
    # 5. Detailed 2021 data for common countries
    df_2021_common = df_2021[df_2021[country_col].astype(str).str.strip().isin(common_countries)]
    df_2021_common.to_excel(writer, sheet_name='Detailed_2021', index=False)

print(f"\n✅ Excel file created successfully: AI_Readiness_Index_2021_2024_Consolidated.xlsx")
print(f"\nFirst 10 rows of merged data:")
print(merged_df.head(10))

print(f"\nBasic Statistics:")
stats = pd.DataFrame({
    '2021': merged_df['Overall_2021'].describe(),
    '2022': merged_df['Overall_2022'].describe(),
    '2023': merged_df['Overall_2023'].describe(),
    '2024': merged_df['Overall_2024'].describe()
})
print(stats)

# Simple visualization
plt.figure(figsize=(12, 6))
years = ['2021', '2022', '2023', '2024']
means = [merged_df[f'Overall_{year}'].mean() for year in years]

plt.plot(years, means, marker='o', linewidth=2, markersize=8)
plt.title('Average AI Readiness Score (2021-2024)', fontsize=14, fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Average Score')
plt.ylim(0, 100)
plt.grid(True, alpha=0.3)

# Add value labels on points
for i, (year, mean_val) in enumerate(zip(years, means)):
    plt.text(i, mean_val + 1, f'{mean_val:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('ai_readiness_trend.png', dpi=300)
plt.show()

print("\n✅ Analysis complete! Check the Excel file and PNG chart.")



  
    # # India
    # ia", 2021, 73.26, 48.04, 47.02, 56.11),
    # ("India", 2022, 75.58, 48.58, 66.83, 63.67),
    # ("India", 2023, 75.18, 49.39, 63.17, 62.58),
    # ("India", 2024, 7("Ind3.32, 50.34, 64.76, 62.81),
    # # Bangladesh
    # ("Bangladesh", 2021, 40.21, 25.10, 42.99, 36.10),
    # ("Bangladesh", 2022, 58.48, 24.80, 47.60, 43.63),
    # ("Bangladesh", 2023, 57.96, 26.07, 54.10, 46.04),
    # ("Bangladesh", 2024, 58.52, 26.26, 56.59, 47.12),
    # # Afghanistan
    # ("Afghanistan", 2021, 25.67, 9.23, 38.23, 24.38),
    # ("Afghanistan", 2022, 14.46, 10.70, 15.21, 13.46),
    # ("Afghanistan", 2023, 13.99, 21.71, 28.11, 21.27),
    # ("Afghanistan", 2024, 8.27, 22.46, 20.05, 16.92),
    # # Bhutan
    # ("Bhutan", 2021, 36.52, 26.15, 39.40, 34.02),
    # ("Bhutan", 2022, 37.33, 26.69, 49.73, 37.91),
    # ("Bhutan", 2023, 36.81, 24.31, 49.52, 36.88),
    # ("Bhutan", 2024, 34.02, 25.58, 56.73, 38.78),
    # # Maldives
    # ("Maldives", 2021, 29.59, 16.08, 66.73, 37.47),
    # ("Maldives", 2022, 33.68, 15.16, 51.76, 33.54),
    # ("Maldives", 2023, 31.71, 20.79, 42.63, 31.71),
    # ("Maldives", 2024, 33.71, 17.22, 43.36, 31.43),
    # # Pakistan
    # ("Pakistan", 2021, 39.58, 35.00, 27.50, 34.03),
    # ("Pakistan", 2022, 44.32, 34.67, 41.67, 40.22),
#     ("Pakistan", 2023, 43.00, 34.07, 49.53, 42.20),
#     ("Pakistan", 2024, 40.61, 36.94, 43.87, 40.47),
#     # Sri Lanka
#     ("Sri Lanka", 2021, 34.84, 32.54, 55.97, 41.12),
#     ("Sri Lanka", 2022, 30.19, 30.92, 47.59, 36.23),
#     ("Sri Lanka", 2023, 42.05, 33.02, 50.60, 41.89),
#     ("Sri Lanka", 2024, 55.04, 32.19, 48.65, 45.29),
# ]

# train_df = pd.DataFrame(country_data, columns=["Country", "Year", "Government", "Technology", "Infrastructure", "TotalScore"])

# --------------------------------------------------
# # 2. Train Model on Combined South Asian Countries
# # --------------------------------------------------

# X_train = train_df[["Government", "Technology", "Infrastructure"]]
# y_train = train_df["TotalScore"]

# model = LinearRegression()
# model.fit(X_train, y_train)

# print("Model Coefficients:", model.coef_)
# print("Intercept:", model.intercept_)

# # ----------------------------------------
# # 3. Load Nepal Test Data (2023 and 2024)
# # ----------------------------------------

# data_nepal_test = {
#     "Year": [2023, 2024],
#     "Government": [31.04, 30.61],
#     "Technology": [24.21, 25.44],
#     "Infrastructure": [37.06, 43.37],
#     "TotalScore": [30.77, 33.14]
# # }

# nepal_df = pd.DataFrame(data_nepal_test)

# X_test = nepal_df[["Government", "Technology", "Infrastructure"]]
# y_test = nepal_df["TotalScore"]

# predicted_test = model.predict(X_test)
# r2 = r2_score(y_test, predicted_test)
# rmse = mean_squared_error(y_test, predicted_test, squared=False)
# mae = mean_absolute_error(y_test, predicted_test)
# mape = mean_absolute_percentage_error(y_test, predicted_test)

# print("\n--- Nepal 2023-2024 Evaluation ---")
# print("Predicted:", predicted_test.round(2))
# print("Actual:", y_test.values)
# print("R2:", round(r2, 4))
# print("RMSE:", round(rmse, 4))
# print("MAE:", round(mae, 4))
# print("MAPE (%):", round(mape * 100, 2))

# ----------------------------------------
# # 4. Predict Nepal 2025 with Simulations
# # ----------------------------------------

# latest_row = nepal_df.iloc[-1][["Government", "Technology", "Infrastructure"]].to_frame().T

# # Simulate change in values
# def simulate_change(row_df, gov_pct=0, tech_pct=0, infra_pct=0):
#     modified = row_df.copy()
#     modified["Government"] *= (1 + gov_pct / 100)
#     modified["Technology"] *= (1 + tech_pct / 100)
#     modified["Infrastructure"] *= (1 + infra_pct / 100)
#     return model.predict(modified)[0]

# scenarios = {
#     "Base (2024 values)": model.predict(latest_row)[0],
#     "+10% Government": simulate_change(latest_row, gov_pct=10),
#     "+10% Technology": simulate_change(latest_row, tech_pct=10),
#     "+10% Infrastructure": simulate_change(latest_row, infra_pct=10),
#     "+10% All": simulate_change(latest_row, gov_pct=10, tech_pct=10, infra_pct=10)
# }

# sim_df = pd.DataFrame(list(scenarios.items()), columns=["Scenario", "Predicted Score"])

# # ----------------------------------------
# # 5. Visualize Simulated Scenarios
# # ----------------------------------------

# plt.figure(figsize=(10,6))
# sns.barplot(data=sim_df, y="Scenario", x="Predicted Score", palette="viridis")
# plt.title("Nepal 2025: AI Readiness under 10% Improvements")
# plt.xlabel("Total Score")
# plt.grid(axis="x", linestyle="--", alpha=0.7)
# plt.tight_layout()
# plt.show()

# print("\n--- 2025 Predictions Summary ---")
# print(sim_df.round(2))
